In [2]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=int("2022"), day=int("19"))

puzzle.header()

# Not Enough Minerals

[Open Website](https://adventofcode.com/2022/day/19)

In [21]:

# helper functions
class Blueprint:
    def __init__(self, id: int, costs: dict):
        self.id = id
        self.costs = costs
        self.useful = {
            "ore": max(self.costs["clay"]["ore"],
                       self.costs["obsidian"]["ore"],
                       self.costs["geode"]["ore"]),
            "clay": self.costs["obsidian"]["clay"],
            "obsidian": self.costs["geode"]["obsidian"],
            "geode": float("inf")
        }

    def __repr__(self):
        return (f"Blueprint(costs={self.costs})")


def domain_from_input(input: InputReader) -> set:
    lines = input.lines_as_str()
    # remove empty lines
    lines = [line for line in lines if line]
    blueprints = set()

    for i in range(0, len(lines), 5):
        costs = {}
        # parse id
        line = lines[i].strip(":").split()
        id = int(line[-1])

        # parse ore robot using pattern
        line = lines[i + 1].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
        }

        # parse clay robot using pattern
        line = lines[i + 2].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
        }

        # parse obsidian robot using pattern
        line = lines[i + 3].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
            line[-4]: int(line[-5]),
        }

        # parse geode robot using pattern
        line = lines[i + 4].strip(".").split()
        costs[line[1]] = {
            line[-1]: int(line[-2]),
            line[-4]: int(line[-5]),
        }

        for type in costs.keys():
            costs[type] = costs[type]

        blueprint = Blueprint(id, costs)
        blueprints.add(blueprint)

    return blueprints


test_input = domain_from_input(puzzle.get_code_block(0))
print(test_input)

{Blueprint(costs={'ore': {'ore': 2}, 'clay': {'ore': 3}, 'obsidian': {'clay': 8, 'ore': 3}, 'geode': {'obsidian': 12, 'ore': 3}}), Blueprint(costs={'ore': {'ore': 4}, 'clay': {'ore': 2}, 'obsidian': {'clay': 14, 'ore': 3}, 'geode': {'obsidian': 7, 'ore': 2}})}


In [22]:
# test case (part 1)
class State:
    def __init__(self, robots: dict = None, resources: dict = None, ignored: list = None):
        self.robots = robots.copy() if robots else {
            "ore": 1,
            "clay": 0,
            "obsidian": 0,
            "geode": 0,
        }
        self.resources = resources.copy() if resources else {
            "ore": 0,
            "clay": 0,
            "obsidian": 0,
            "geode": 0
        }
        self.ignored = ignored.copy() if ignored else []

    def copy(self) -> "State":
        return State(self.robots, self.resources, self.ignored)

    def __gt__(self, other):
        return self.resources["geode"] > other.resources["geode"]

    def __repr__(self):
        return f"{{robots: {self.robots}, resources: {self.resources}}}"


def evaluate_options(
        blueprint: Blueprint,
        prior_states: list[State],
        timelimit: int = 26
) -> [tuple[int, list]]:
    time_remaining = timelimit - len(prior_states)
    curr_state = prior_states[-1]

    # determine options for what to build in the next state
    options: list[str] = []
    if time_remaining >= 0:
        # look for something affordable and useful and not ignored last time
        for robot, cost in blueprint.costs.items():
            if (curr_state.robots[robot] < blueprint.useful[robot]
                    and all(curr_state.resources[k] >= v for k, v in cost.items())
                    and robot not in curr_state.ignored):
                options.append(robot)

        # geodes before anything else, don't bother with other types at the end
        if "geode" in options:
            options = ["geode"]
        elif time_remaining < 1:
            options = []
        else:
            # cutting off plans that build resources more than 2 phases back
            if ((curr_state.robots["clay"] > 3 or curr_state.robots["obsidian"]
                 or "obsidian" in options) and "ore" in options):
                options.remove("ore")
            if ((curr_state.robots["obsidian"] > 3 or curr_state.robots["geode"]
                 or "geode" in options) and "clay" in options):
                options.remove("clay")

        # add new resources
        next_state = curr_state.copy()
        for r, n in next_state.robots.items():
            next_state.resources[r] += n

        # the 'do nothing' option
        next_state.ignored += options
        results = [evaluate_options(blueprint, prior_states + [next_state], timelimit)]

        # the rest of the options
        for opt in options:
            next_state_opt = next_state.copy()
            next_state_opt.ignored = []
            next_state_opt.robots[opt] += 1
            for r, n in blueprint.costs[opt].items():
                next_state_opt.resources[r] -= n
            results.append(
                evaluate_options(blueprint, prior_states + [next_state_opt], timelimit)
            )

        return max(results)

    return prior_states[-1].resources["geode"], prior_states


def part_1(reader: InputReader, debug: bool) -> int:
    blueprints = domain_from_input(reader)
    result = 0
    for bp in blueprints:
        r = evaluate_options(bp, [State()], 24)
        result += r[0] * bp.id
    return result


result = part_1(puzzle.example(0), True)
print(result)
assert result == 33

33


In [23]:
# real case (part 1)
def generate_input() -> InputReader:
    lines = puzzle.input().lines_as_str()
    final_lines = []
    # add newline before the word Each.
    for line in lines:
        for next in line.replace("Each", "\nEach").split("\n"):
            final_lines.append(next.strip())
    return InputReader("\n".join(final_lines))


result = part_1(generate_input(), False)
print(result)
assert result == 1144

1144


In [ ]:
# test case (part 2)
def part_2(reader: InputReader, debug: bool) -> int:
    blueprints = list(domain_from_input(reader))
    blueprints.sort(key=lambda x: x.id)
    if len(blueprints) > 3:
        blueprints = blueprints[:3]
    result = 1
    for blueprint in blueprints:
        r = evaluate_options(blueprint, [State()], 32)
        result *= r[0]
    return result


result = part_2(puzzle.example(0), True)
print(result)
assert result == 62

In [25]:
# real case (part 2)
input = generate_input()
result = part_2(input, False)
print(result)
assert result == 19440

19440


In [26]:
# print easters eggs
puzzle.print_easter_eggs()

## Easter Eggs

<span title="If You Give A Mouse An Ore-Collecting Robot">kickstart</span> (If You Give A Mouse An Ore-Collecting Robot)

In [3]:
import re
import numpy as np


def parse19():
    data = [[int(i) for i in re.findall("\d+", l)] for l in puzzle.input().lines_as_str()]
    # saving robot costs as array of resources [ore, clay, obsidian, geodes]
    bps = []
    for d in data:
        ore_rob_cost = np.array([d[1], 0, 0, 0])
        cla_rob_cost = np.array([d[2], 0, 0, 0])
        obs_rob_cost = np.array([d[3], d[4], 0, 0])
        geo_rob_cost = np.array([d[5], 0, d[6], 0])
        bps.append([ore_rob_cost, cla_rob_cost, obs_rob_cost, geo_rob_cost])
    return bps


from queue import Queue


def hash_state(state):
    time, robots, resources = state
    h = str(time) + "_"
    for r in robots:
        h += "_" + str(r)
    h += "_"
    for r in resources:
        h += "_" + str(r)
    return h


def max_geodes(blueprint, max_time=24):
    # compute maximum amount of each resource needed to build any robot
    max_res = np.zeros(4, dtype=int)
    for cost in blueprint:
        for resource_type in range(len(cost)):
            if cost[resource_type] > max_res[resource_type]:
                max_res[resource_type] = cost[resource_type]

    # initial state
    robots = np.array([1, 0, 0, 0])
    resources = np.array([0, 0, 0, 0])
    time = 0
    start = (time, robots, resources)

    # BFS-like search of state evolutions
    states = {hash_state(start)}
    geodes_max = 0
    queue = Queue()
    queue.put(start)

    while not queue.empty():

        # get state
        time, robots, resources = queue.get()

        # compute geodes from current state at end time
        geodes_this_state = resources[3] + robots[3] * (max_time - time)
        if geodes_this_state > geodes_max:
            geodes_max = geodes_this_state

        # fast-forward in time to a new state where a new robot can be built
        # given collected resources and what could be built in that time interval
        # doing this for all possible robots
        for robot_type in range(len(blueprint)):  # index of robot to be built
            cost = blueprint[robot_type]
            time_needed = [0, 0, 0, 0]  # time needed to gather resources to build robot
            for resource_type in range(len(cost)):  # needed resources
                if cost[resource_type]:  # resource is needed to build robot
                    if cost[resource_type] <= resources[
                        resource_type]:  # state already has enough of this resource to build robot
                        continue
                    else:  # compute time needed to produce resource
                        if robots[resource_type]:  # have robot(s) to produce resource
                            time_needed[resource_type] = (cost[resource_type] - resources[resource_type]) // robots[
                                resource_type] + int(
                                (cost[resource_type] - resources[resource_type]) % robots[resource_type] > 0)
                        else:  # no robot to collect resource, storing a too-large time value to reject construction
                            time_needed[resource_type] = max_time + 1
            time_needed_to_collect = max(time_needed)  # choosing time from most time-consuming resource
            if time + time_needed_to_collect + 1 + 1 <= max_time:  # resources can be gathered and robot can be built in available time
                # and new robot will have time to do something (+1 minute), otherwise useless
                # collect resources with initially available robots, spend to build new robot
                resources_new = resources + (time_needed_to_collect + 1) * robots - cost
                # build new robot
                robots_new = np.copy(robots)
                robots_new[robot_type] += 1

                # OPTIMISATIONS: The code converges to the correct results even without these optimisations,
                # but the search space becomes very large and the execution time very slow...

                # 1) If it takes N resources to build a robot, it's uses to have M>N robots collecting that resource,
                # so I can speed-up the process by avoiding to re-enque states with too many useless robots.
                # This is enough to solve Part 1 in a decent time
                if not (robots_new <= max_res)[:3].all():  # do not consider geodes
                    continue

                # 2) Let's imagine that from next round only geodes robots will be added to this state (regardless
                # of whether this is possible in terms of resources). If even in this overoptimistic conditions the
                # state cannot produce more geodes than the current maximum, it's useless to re-enque this state
                time_left = max_time - (time + time_needed_to_collect + 1)
                geodes_new_ideal = (time_left - 1) * time_left // 2  # triangular number for timeleft-1, since geodes
                # built at last minute cannot build anything
                geodes_final_ideal = resources_new[3] + time_left * robots_new[3] + geodes_new_ideal
                if geodes_final_ideal <= geodes_max:
                    continue

                # re-enque new state
                state_new = (time + time_needed_to_collect + 1, robots_new, resources_new)
                h = hash_state(state_new)
                if h not in states:
                    queue.put(state_new)
                    states.add(h)

    return geodes_max


def part2():
    bps = parse19()
    p = 1
    print("| Blueprint | Geodes (32) |")
    print("|-----------+-------------|")
    for i, bp in enumerate(bps):
        g = max_geodes(bp, 32)
        print("| {:9d} | {:11d} | ".format(i + 1, g))
        p *= g
        if i + 1 == 3:
            break
    print("\n Product of max geodes: {}".format(p))
    return p


print("\nPart 2")
part2()

<>:5: SyntaxWarning: invalid escape sequence '\d'
<>:5: SyntaxWarning: invalid escape sequence '\d'
/var/folders/by/zlrp4lzn0q506hwfkggql0_w0000gn/T/ipykernel_3191/3245722533.py:5: SyntaxWarning: invalid escape sequence '\d'
  data = [ [ int(i) for i in re.findall("\d+",l) ] for l in puzzle.input().lines_as_str() ]


Part 1
| Blueprint | Geodes (24) | Quality |
|-----------+-------------+---------|
|         1 |           0 |       0 | 
|         2 |           9 |      18 | 
|         3 |           3 |       9 | 
|         4 |           3 |      12 | 
|         5 |           0 |       0 | 
|         6 |           0 |       0 | 
|         7 |           0 |       0 | 
|         8 |           1 |       8 | 
|         9 |           3 |      27 | 
|        10 |           9 |      90 | 
|        11 |           1 |      11 | 
|        12 |           0 |       0 | 
|        13 |           1 |      13 | 
|        14 |           1 |      14 | 
|        15 |           0 |       0 | 
|        16 |           2 |      32 | 
|        17 |           2 |      34 | 
|        18 |           0 |       0 | 
|        19 |           1 |      19 | 
|        20 |           4 |      80 | 
|        21 |           4 |      84 | 
|        22 |           3 |      66 | 
|        23 |           1 |      23 | 
|        24 |       

np.int64(19980)